# **Streamlit**

In [ ]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler

# --------------------------------------------------
# PAGE CONFIG
# --------------------------------------------------
st.set_page_config(page_title="Hybrid Movie Recommender", layout="wide")
st.title("🎬 Intelligent Hybrid Movie Recommendation System")
st.caption("Content-based • Collaborative Filtering • Matrix Factorization")

# --------------------------------------------------
# LOAD ARTIFACTS
# --------------------------------------------------
@st.cache_resource
def load_artifacts():
    tmdb = pickle.load(open("artifacts/tmdb_df.pkl", "rb"))
    title_col = pickle.load(open("artifacts/title_col.pkl", "rb"))
    content_similarity = pickle.load(open("artifacts/content_similarity.pkl", "rb"))

    movies_ml = pickle.load(open("artifacts/movies_ml.pkl", "rb"))
    item_sim_df = pickle.load(open("artifacts/item_similarity.pkl", "rb"))

    user_latent = pickle.load(open("artifacts/user_latent.pkl", "rb"))
    item_latent = pickle.load(open("artifacts/item_latent.pkl", "rb"))

    popularity = pickle.load(open("artifacts/popularity.pkl", "rb"))
    return tmdb, title_col, content_similarity, movies_ml, item_sim_df, user_latent, item_latent, popularity

tmdb, title_col, content_similarity, movies_ml, item_sim_df, user_latent, item_latent, popularity = load_artifacts()

tmdb_index = pd.Series(tmdb.index, index=tmdb[title_col])
movieid_to_title = dict(zip(movies_ml["movieId"], movies_ml["title"]))
scaler = MinMaxScaler()

# --------------------------------------------------
# POSTER FETCH
# --------------------------------------------------
TMDB_API_KEY = st.secrets["TMDB_API_KEY"]

def fetch_poster(title):
    try:
        url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={title}"
        data = requests.get(url).json()
        if data["results"]:
            poster = data["results"][0].get("poster_path")
            if poster:
                return f"https://image.tmdb.org/t/p/w500/{poster}"
    except:
        pass
    return "https://placehold.co/300x450?text=No+Poster"

# --------------------------------------------------
# MF SCORE
# --------------------------------------------------
def mf_scores(user_index):
    scores = np.dot(user_latent[user_index], item_latent)
    return scaler.fit_transform(scores.reshape(-1, 1)).flatten()

# --------------------------------------------------
# HYBRID RECOMMENDER (WITH EXPLANATIONS)
# --------------------------------------------------
def hybrid_recommend(
    user_index,
    seed_movie_title,
    k,
    w_content,
    w_item,
    w_mf,
    w_pop
):
    idx = tmdb_index[seed_movie_title]
    content_scores = sorted(
        list(enumerate(content_similarity[idx])),
        key=lambda x: x[1],
        reverse=True
    )[1:50]

    mf_score_array = mf_scores(user_index)
    results = []

    for tmdb_idx, c_score in content_scores:
        title = tmdb.iloc[tmdb_idx][title_col]
        ml_match = movies_ml[movies_ml["title"].str.contains(title, case=False)]

        if ml_match.empty:
            continue

        movie_id = ml_match.iloc[0]["movieId"]
        item_score = item_sim_df.loc[movie_id].mean()
        mf_score = mf_score_array[item_sim_df.columns.get_loc(movie_id)]
        pop_score = popularity.get(movie_id, 0)

        final_score = (
            w_content * c_score +
            w_item * item_score +
            w_mf * mf_score +
            w_pop * pop_score
        )

        explanation = []
        if c_score > 0.3:
            explanation.append("similar content")
        if item_score > 0.3:
            explanation.append("similar user behavior")
        if mf_score > 0.3:
            explanation.append("latent preference match")
        if pop_score > 0.2:
            explanation.append("popular among users")

        results.append((movie_id, final_score, explanation))

    results = sorted(results, key=lambda x: x[1], reverse=True)[:k]
    return results

# --------------------------------------------------
# SIDEBAR CONTROLS
# --------------------------------------------------
st.sidebar.header("🔧 Controls")

user_index = st.sidebar.number_input(
    "User Index",
    min_value=0,
    max_value=user_latent.shape[0] - 1,
    value=10
)

seed_movie = st.sidebar.selectbox(
    "Seed Movie",
    sorted(tmdb[title_col].unique())
)

k = st.sidebar.slider("Recommendations", 5, 15, 10)

st.sidebar.subheader("Model Weights")
w_content = st.sidebar.slider("Content", 0.0, 1.0, 0.30)
w_item = st.sidebar.slider("Collaborative", 0.0, 1.0, 0.30)
w_mf = st.sidebar.slider("Latent Factors", 0.0, 1.0, 0.25)
w_pop = st.sidebar.slider("Popularity", 0.0, 1.0, 0.15)

# --------------------------------------------------
# RUN
# --------------------------------------------------
if st.sidebar.button("🎯 Recommend"):
    with st.spinner("Thinking like Netflix..."):
        recs = hybrid_recommend(
            user_index,
            seed_movie,
            k,
            w_content,
            w_item,
            w_mf,
            w_pop
        )

    cols = st.columns(5)
    for i, (movie_id, score, why) in enumerate(recs):
        with cols[i % 5]:
            title = movieid_to_title[movie_id]
            st.image(fetch_poster(title))
            st.markdown(f"**{title}**")
            st.caption("Because of: " + ", ".join(why))
            st.caption(f"Score: {score:.2f}")

# --------------------------------------------------
# FOOTER
# --------------------------------------------------
st.markdown("---")
st.caption("Hybrid Recommender • Content + CF + MF + Popularity • Explainable AI")
